# TensorBoard

With TensorBoard it's possible to visualize and investigate the machine learning model.
- Visualize the graph
- Track for example metrics like loss and accuracy during hyperparameter training
- and many more see https://www.tensorflow.org/tensorboard/

In this tutorial we will see how a hyperparameter training is done to 

In [ ]:
import tensorflow as tf
import numpy as np

from utils.model import (
    CustomCNNV1Model
)

from utils.utils import (
    calc_gaussians,
    gen_gaussian_kernel_v1_1D,
    load_data_cnn,
    plot_gaussian_weights,
    plot_derivative_energy
)

from tensorboard.plugins.hparams import api as hp

# import data
data_path = 'data/orbital_free_DFT/'

kinetic_train, kinetic_derivativ_train, density_train = load_data_cnn(
    path=data_path,
    data_name='M=100_training_data.hdf5'
)

kinetic_test, kinetic_derivativ_test, density_test = load_data_cnn(
    path=data_path,
    data_name='test_data.hdf5'
)

training_dataset = tf.data.Dataset.from_tensor_slices(
    (
        density_train.astype(np.float32),
        {'T': kinetic_train.astype(np.float32),
        'dT_dn': kinetic_derivativ_train.astype(np.float32)}
    )
).batch(100).repeat(10)

In [ ]:
# We can define the range/valus and hyperparameters we wabnt to try out.
HP_LR = hp.HParam('lr', hp.Discrete([0.01, 0.001])) # learning rate
HP_KS = hp.HParam('ks', hp.Discrete([50, 100, 150])) # kernel size
HP_FL = hp.HParam('fl', hp.Discrete([4, 8, 16, 32])) # filter length
HP_AF = hp.HParam('af', hp.Discrete(['softplus', 'sigmoid'])) # activation function

# The metrics we want to track
METRIC_LOSS = 'loss'
METRIC_T_loss = 'T_loss'
METRIC_dT_dn_loss = 'dT_dn_loss'
METRIC_T_mae = 'T_mae'
METRIC_dT_dn_mae = 'dT_dn_mae'

# Define path were results/logging of hyperparameter search will be saved
log_dir = 'logs/hyperparam_search/'

with tf.summary.create_file_writer(log_dir).as_default():
    # Set the hyperparameters and metrics here
    hp.hparams_config(
    hparams=[HP_LR, HP_KS, HP_FL, HP_AF],
    metrics=[
        hp.Metric(METRIC_LOSS, display_name='loss'),
        hp.Metric(METRIC_T_loss, display_name='T_loss'),
        hp.Metric(METRIC_dT_dn_loss, display_name='dT_dn_loss'),
        hp.Metric(METRIC_T_mae, display_name='T_mae'),
        hp.Metric(METRIC_dT_dn_mae, display_name='dT_dn_mae')
    ],
)

Define a method where the model is initialized/build/compiled/fitted.
This method is going to be called for each hyperparameter combination
and we return the metrics we want to track.

In [ ]:
def train_test_model(hparams):
    # the hyperparamters for one run are passed as a list

    # Now, we initialize/build/compile the model
    model = CustomCNNV1Model(
        filter_size=hparams[HP_FL],
        kernel_size=hparams[HP_KS],
        layer_length=3,
        dx=0.002,
        create_continuous_kernel=gen_gaussian_kernel_v1_1D,
        kernel_regularizer=tf.keras.regularizers.l2(0.00025),
        activation=hparams[HP_AF]
    )

    model.build(input_shape=(None, 500, 1))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hparams[HP_LR], amsgrad=False
        ),
        loss={'T': 'mse', 'dT_dn': 'mse'},
        loss_weights={'T': 0.2, 'dT_dn': 1.0}, # scale the loss in T by 0.2
        metrics={'T': ['mae'], 'dT_dn': ['mae']}
    )
    
    model.fit(
        training_dataset,
        epochs=10,
        verbose=2, # With 0, there no print of loss. With 1, the print is with loss and a loading bar (batch/epoch), with 2 print with loss 
        validation_data=(
            density_test, {'T': kinetic_test, 'dT_dn': kinetic_derivativ_test}
        ),
        validation_freq=2
    )
    
    loss = model.history.history['val_loss'][-1]
    val_T_loss = model.history.history['val_T_loss'][-1]
    val_dT_dn_loss = model.history.history['val_dT_dn_loss'][-1]
    val_T_mae = model.history.history['val_T_mae'][-1]
    val_dT_dn_mae = model.history.history['val_dT_dn_mae'][-1]
    return loss, val_T_loss, val_dT_dn_loss, val_T_mae, val_dT_dn_mae

In run we call the train_test_model and logg the metrics

In [ ]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        loss, val_T_loss, val_dT_dn_loss, val_T_mae, val_dT_dn_mae = train_test_model(hparams)
        tf.summary.scalar(METRIC_LOSS, loss, step=1)
        tf.summary.scalar(METRIC_T_loss, val_T_loss, step=1)
        tf.summary.scalar(METRIC_dT_dn_loss, val_dT_dn_loss, step=1)
        tf.summary.scalar(METRIC_T_mae, val_T_mae, step=1)
        tf.summary.scalar(METRIC_dT_dn_mae, val_dT_dn_mae, step=1)

In [ ]:
session_num = 0
# Don't forget to set a seed
seed = 0
tf.random.set_seed(seed)

# For all hyperparameters we execute the run method 
for lr in HP_LR.domain.values:
    for fl_size in HP_KS.domain.values:
        for kr_length in HP_FL.domain.values:
            for activatuion in HP_AF.domain.values:
                hparams = {
                    HP_LR: lr,
                    HP_KS: fl_size,
                    HP_FL: kr_length,
                    HP_AF: activatuion
                }
                run_name = "run-%d" % session_num
                print('--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                run(log_dir + run_name, hparams)
                session_num += 1